In [1]:
import pandas as pd
import numpy as np

In [2]:
paperIds = pd.read_csv('/home/lisi/impact-prediction/data/paperIds-1975-2005-2015-2.tsv', sep='\t', names=['paperIds'])

In [3]:
# this file has a lot of sparse rows so we open it in this way
with open('/home/lisi/impact-prediction/data/paperHistories-1975-2005-2015-2.tsv', 'r') as f:
    lines = [row.split() for row in f.read().split("\n") if row]

In [4]:
paperIds['citations'] = lines

In [5]:
respones = pd.read_csv('/home/lisi/impact-prediction/data/paperResponses-1975-2005-2015-2.tsv', sep='\t')

In [6]:
respones['paperIds'] = paperIds.paperIds.values

In [7]:
paperIds.citations = paperIds.citations.apply(lambda x: x[-5:])

In [8]:
paperIds.citations = paperIds.citations.apply(lambda x: np.nan if len(x) < 5 else x)

In [9]:
paperIds = pd.merge(paperIds, respones, how='left', on='paperIds')

In [10]:
paperIds = paperIds.dropna().reset_index(drop=True)

In [11]:
ids = paperIds.paperIds.values
c6 = paperIds.total_citations_in_0.values
c7 = paperIds.total_citations_in_1.values
c8 = paperIds.total_citations_in_2.values
c9 = paperIds.total_citations_in_3.values
c10 = paperIds.total_citations_in_4.values
c11 = paperIds.total_citations_in_5.values
c12 = paperIds.total_citations_in_6.values
c13 = paperIds.total_citations_in_7.values
c14 = paperIds.total_citations_in_8.values
c15 = paperIds.total_citations_in_9.values
paperIds = pd.DataFrame(paperIds["citations"].to_list(), columns=['citations_{}'.format(j) for j in range(1,6)])
paperIds['paperIds'] = ids
paperIds['citations_6'] = c6
paperIds['citations_7'] = c7
paperIds['citations_8'] = c8
paperIds['citations_9'] = c9
paperIds['citations_10'] = c10
paperIds['citations_11'] = c11
paperIds['citations_12'] = c12
paperIds['citations_13'] = c13
paperIds['citations_14'] = c14
paperIds['citations_15'] = c15

In [12]:
# we read the citation graph and make a csv file
with open('/home/lisi/impact-prediction/data/paperIdToPaperData-1975-2015.json', 'r',encoding='utf-8') as j:
        d = j.read().replace('false', '0').replace('true', '1').splitlines()

In [13]:
# string to dictionary
good = []
bad = [] # there are some authors that have special characters that utf-8 or utf-16 cant solve
for i in d:
    try:
        good.append(eval(i))
    except:
        bad.append(i)

In [14]:
# manually added the bad rows
add_bad = {'id':['ca9ba78b3e268ec9b58ffa69524e6f35c0314fcc', 
                 '30af66277400354f88d0a5de0feaab9fb5278705', 
                '61afe58c68b91d20a65bb4166cfceb501745c134', 
                '6d46189d2040a01ccdb4236b0095361294b1a8bf'],
          'year':[2007, 2007, 2006, 2004],
          'venue':['','','','']}

In [15]:
metadata = pd.DataFrame.from_dict(good, orient='columns') # list to dataframe
metadata = metadata.drop(['citations-in-year', 'authors', 'is-survey'],axis=1)
metadata = metadata.append(pd.DataFrame(add_bad), ignore_index=True) # add the bad rows

In [16]:
metadata['venue'].replace('', np.nan, inplace=True)

In [17]:
metadata = metadata.dropna().reset_index(drop=True)

In [18]:
metadata = pd.merge(metadata, paperIds, left_on=['id'], right_on=['paperIds'], how='right')

In [ ]:
# metadata = metadata.drop(['id', 'paperIds'], axis=1)

In [19]:
xtest = pd.read_csv('/home/lisi/Desktop/Laci/LaszloModel/test_set_ids.csv')
xtrain = pd.read_csv('/home/lisi/Desktop/Laci/LaszloModel/train_set_ids.csv')

In [22]:
X_test = metadata[metadata.id.isin(list(xtest.id.values))].drop(['paperIds'],axis=1)
X_train = metadata[metadata.id.isin(list(xtrain.id.values))].drop(['paperIds'],axis=1)

In [25]:
X_train.to_csv('rnn_x_train.csv', index=False)
X_test.to_csv('rnn_x_test.csv', index=False)

In [26]:
X_test

,year,id,venue,citations_1,citations_2,citations_3,citations_4,citations_5,citations_6,citations_7,citations_8,citations_9,citations_10,citations_11,citations_12,citations_13,citations_14,citations_15
12,1988.0,1b0d090a6518235977d0ecf532329fbdb9fafb56,compgeom,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1
39,1997.0,dd50d87c777cecfbe0e2994262b6e08c94984d4f,orl,0,0,0,1,1,1,1,1,2,3,3,3,3,3,3
40,1996.0,1e70d2eb80eba7a3ec384f6d0f41612a10e7e38d,ht,6,7,7,7,7,8,9,9,9,9,9,9,9,9,9
72,2001.0,41c19582e260f268524b87b95b58b2b4d8712fee,cn,6,10,12,16,17,20,23,24,25,26,27,28,31,32,33
86,1997.0,df6ddb7456d80689970e78065684add1eb6bda80,jcmc,3,4,7,9,9,10,10,10,11,11,11,11,11,12,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
691475,1997.0,9c7025c8b0cf4bec8877e1b9cd9c03aa21eab7ab,sas,2,3,3,3,4,5,5,5,5,5,5,5,5,5,5
691503,1999.0,2da3596196ea3b1a932b02503d44d3f218b38c6e,icip,0,0,1,2,2,3,4,4,4,4,4,4,4,5,5
691538,1997.0,4cf4076601ee77c6ce00183e6806591cb759e1cd,arvlsi,2,2,2,3,3,3,3,4,4,4,4,5,5,5,5
691559,2000.0,dc7773082466cc2f2d11f69b8d9191df41ac244f,cvpr,0,0,0,2,2,2,2,2,2,2,2,2,2,2,2
